In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sep 12 2018
@author: Fernanda Farinelli

Resource Information
    Response formats --> JSON
    Requires authentication? --> Yes
    Rate limited? --> Yes
    Requests / 15-min window (user auth) --> 180
    Requests / 15-min window (app auth) --> 450


"""

'\nCreated on Sep 12 2018\n@author: Fernanda Farinelli\n\nResource Information\n    Response formats --> JSON\n    Requires authentication? --> Yes\n    Rate limited? --> Yes\n    Requests / 15-min window (user auth) --> 180\n    Requests / 15-min window (app auth) --> 450\n\n\n'

In [2]:
#Globals variables and constants

consumer_key = 'rWxMBAKB6IgXNVrsXkisVebMU'
consumer_secret = 'xRFc47WcbcgoZYb26OHfMXakda7UFIQMvZw2WftbOsAXpBRKMS'
access_token_key = '1028301224519589889-hrC4g78bb0PJ9RiEbusi657it517wf'
access_token_secret = 'hhwamERYlZUJuLQwWnJtvWvqpB5hFCyNBOCiDVDrT5xIL'
#filename = "C://Users//unesp//ProjetoOpLaDyn//inputs//twitter_tokens_pt.txt"
fileInput = "C://Users//unesp//ProjetoOpLaDyn//inputs//twitter_tokens_Marielle_pt.txt"
folderOutput = 'C:\\Users\\unesp\\ProjetoOpLaDyn\\outputs\\'

#Using the standard search endpoint
resourceURL = "https://api.twitter.com/1.1/search/tweets.json"
queryURL = "?l=pt&q=%23Marielle%20OR%20%23MarielleFranco%20OR%20%23MariellePresente%20OR%20%23Justi%C3%A7aParaMarielleEAnderson%20OR%20%23QuemMatouMarielle%20since%3A2018-09-01%20until%3A2018-09-02&src=typd"

completeQueryURL = "https://api.twitter.com/1.1/search/tweets.json?l=pt&q=%23Marielle%20OR%20%23MarielleFranco%20OR%20%23MariellePresente%20OR%20%23Justi%C3%A7aParaMarielleEAnderson%20OR%20%23QuemMatouMarielle%20since%3A2018-09-01%20until%3A2018-09-02&src=typd"



In [3]:
import tweepy #https://github.com/tweepy/tweepy
import csv
import json
import time

In [4]:
def get_authorize_twitter():
    global consumer_key, consumer_secret, access_token_key, access_token_secret
    
    #authorize twitter, initialize tweepy
    # Creating the authentication object
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    # Setting your access token and secret
    auth.set_access_token(access_token_key, access_token_secret)
    # Creating the API object while passing in auth information
    return tweepy.API(auth)

def writeJSON(alltweets,timestr):
    filename = folderOutput + "tweetsKeywordsCaseMarielle_pt_" + timestr
    #write tweet objects to JSON
    file = open(filename + '.json','w')
    print ("Writing tweet objects to JSON please wait...")
    for status in alltweets:
        json.dump(status._json,file,sort_keys = True,indent = 4)
    #close the file
    print ("Done with JSON")
    file.close()

def writeCSV(alltweets,timestr):
    #transform the tweepy tweets into a 2D array that will populate the csv	
    print ("...writing csv - step 1")
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    filename = folderOutput + "tweetsKeywordsCaseMarielle_pt_" + timestr
    #write the csv	
    with open(filename + '.csv', 'w') as f:
        print ("...writing csv")        
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])        
        writer.writerows(outtweets)

def get_all_tweets(api):
    global resourceURL,queryURL
    count=1000
    rpp=100
    fetched_tweets = ''
    #alltweets = []
    
    #fetched_tweets =  tweepy.Cursor(api.search,q=queryURL,lang="pt",rpp=100,count=1000,tweet_mode='extended')
    #print('fetched_tweets:',len(fetched_tweets))

    max_tweets = 1000
    searched_tweets = [status for status in tweepy.Cursor(api.search,q=queryURL,lang="pt",rpp=rpp,count=count,tweet_mode='extended').items(max_tweets)]
    searched_tweets = []
    last_id = -1
    while len(searched_tweets) < max_tweets:
        count = max_tweets - len(searched_tweets)
        try:
            print('queryURL:',queryURL)
            new_tweets = api.search(q=queryURL,lang="pt",rpp=rpp,count=count, max_id=str(last_id - 1),tweet_mode='extended')
            print('new_tweets:',len(new_tweets))
            if not new_tweets:
                break
            searched_tweets.extend(new_tweets)
            last_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # depending on TweepError.code, one may want to retry or wait
            # to keep things simple, we will give up on an error
            break    
    
    print('searched_tweets:',len(searched_tweets))
    #preparing file to write
    timestr = time.strftime("%Y%m%d-%H%M%S")
    print(timestr)
    
    writeJSON(searched_tweets,timestr)
    
    
    

In [5]:
if __name__ == '__main__':
    
    api = get_authorize_twitter()
    get_all_tweets(api)
    

queryURL: ?l=pt&q=%23Marielle%20OR%20%23MarielleFranco%20OR%20%23MariellePresente%20OR%20%23Justi%C3%A7aParaMarielleEAnderson%20OR%20%23QuemMatouMarielle%20since%3A2018-09-01%20until%3A2018-09-02&src=typd
new_tweets: 0
searched_tweets: 0
20180912-162045
Writing tweet objects to JSON please wait...
Done with JSON
